In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from tqdm import tqdm
from numpy.fft import fft, fftshift
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm

import warnings
warnings.filterwarnings(action='ignore')

# print(tf.__version__) 

#### Data loading

In [2]:
test=pd.read_csv('./2win.csv')

# submission=pd.read_csv('./sample_submission.csv')

# pd.options.display.max_columns=50

In [3]:
test.shape

(88, 6)

In [4]:
test

,com,quest,answer,length,result,date
0,삼성카드,삼성카드를 지원한 이유와 입사 후 회사에서 이루고 싶은 꿈을 기술하시오,삼성카드는 카드사 중에서 유일한 상장사라는 점과 ‘미래에 카드는 쓰지 않을 것이다’...,700,x,09월 13일
1,삼성카드,"본인의 성장과정을 간략히 기술하되 현재의 자신에게 가장 큰 영향을 끼친 사건, 인물...","지금까지의 김명환을 설명하자면 ‘책임’, ‘사람’ 두 단어를 빼놓을 수 없습니다. ...",1500,x,09월 13일
2,삼성카드,최근 사회이슈 중 중요하다고 생각되는 한가지를 선택하고 이에 관한 자신의 견해를 기...,공공데이터 인턴을 수행할 때 처음으로 접하게된 ‘마이데이터’라는 개념은 어느새 법으...,1000,x,09월 13일
3,삼성카드,지원 직군과 관련한 성공경험 또는 입사 후 직무 관련 성과창출을 위해 본인이 지금까...,[밤낮으로 노력하여 우수상을 수상]\n 데이터공부를 위해 통계와 데이터 분석 관련...,1000,x,09월 13일
4,현대해상,"본인의 강점, 가치관 등을 Keyword 형태로 자유롭게 기술하여 주시기 바랍니다.",▶ 감성사전을 통한 감성분류 정확도 94%달성\n\nㅇ에서 뉴스 빅데이터 분석을 통...,1000,x,09월 27일
...,...,...,...,...,...,...
83,한국투자신탁운용,지원분야에 본인이 적합한 이유를 증명하시오.,▶신기술을 접목하는 데이터 전문가◀\n\n빅데이터의 수집부터 머신러닝 기반 활용 및...,500,NaN,10월 28일
84,한국투자신탁운용,"당신의 인생계획(Life Plan)에서 꿈은 무엇이고, 그 꿈을 이루기 위해 회사가...",▶사람 맞춤형 데이터 사이언티스트◀\n\n제 꿈은 사람들을 행복하게 해주는 일을 하...,500,NaN,10월 28일
85,SSG.COM,당사에 지원한 이유와 입사를 위해 어떤 노력을 하였는지 구체적으로 기술하시오,▶사람 맞춤형 데이터 사이언티스트◀\n\n제 꿈은 사람들을 행복하게 해주는 일을 하...,1000,NaN,10월 28일
86,SSG.COM,지원한 직군에서 구체적으로 하고 싶은 일과 본인이 그 일을 남들보다 잘할 수 있는 ...,저는 다음과 같은 맞춤형 서비스를 기획해보고 싶습니다.\n* 소비패턴 분석으로 맞춤...,1000,NaN,10월 28일


In [10]:
test[test['quest'].str.contains('가장')]

,com,quest,answer,length,result,date
1,삼성카드,"본인의 성장과정을 간략히 기술하되 현재의 자신에게 가장 큰 영향을 끼친 사건, 인물...","지금까지의 김명환을 설명하자면 ‘책임’, ‘사람’ 두 단어를 빼놓을 수 없습니다. ...",1500,x,09월 13일
25,한화시스템,"협업을 통해 좋은 성과를 달성했던 사례와, 실패했던 사례를 한가지씩 말씀해주십시오....",▶학생회 최고 참여율 달성◀\n\n학부 시절 학과 학생회장을 맡아 학생회를 운영한 ...,1000,x,10월 04일
33,넷마블네오,"타인과 협력하는 과정에서 가장 어려웠던 일을 사례를 들어 설명하고, 어려움을 어떻게...",대학 졸업후에 해병대 장교로 임관하여 서해최북단 백령도에서 근무했습니다. 하계 IB...,1000,x,10월 11일
36,신영증권,"성장과정을 현재의 자신에게 가장 큰 영향을 끼친 사건, 인물 등을 중심으로 서술해주...","▶최전방 통신체계를 책임지다◀\n\n지난 2018년, 저는 해병대 장교로 임관했습니...",500,NaN,10월 18일
37,신영증권,가장 열정을 가지고 도전하여 성취한 경험에 대해 서술해주십시오.,▶주식도메인 뉴스분류모델 개발◀\n\n산학프로젝트에서 신규투자자들의 투자의사결정을 ...,500,NaN,10월 18일
45,이베스트투자증권,"자신이 지원한 부문에 가장 필요한 것을 무엇이라고 생각하며, 이것을 갖추기 위한 자...",▶검증된 데이터 리터러시◀\n\n빅데이터의 수집부터 머신러닝 기반 활용 및 시각화까...,500,x,10월 18일
66,우리카드,우리카드가 속한 산업 생태계는 급속한 변화가 이루어지고 있습니다. 이에 대응하기 위...,▶ 학생 목소리에 집중한 정책으로 참여율 70% 달성 ◀\n\n학부 시절 저희 학과...,800,NaN,10월 25일
69,우리카드,지금까지 사용해본 금융 관련 각종 서비스 중 가장 혁신적이라고 생각했던 서비스와 그...,"▶ 당연했던 불편함을 해소해준 서비스 ◀\n\n대학생 시절, 학교에서 신입생을 위한...",800,NaN,10월 25일
77,교보생명보험,지금까지 가장 도전적인 목표를 정하고 노력한 경험이 있다면 작성하여 주십시오\n또한...,▲ 주식도메인 뉴스분류모델 개발\n\nㅇ에서 뉴스 빅데이터 분석을 통한 주가변동성 ...,1000,NaN,10월 26일
87,SSG.COM,학업 외 가장 열정적이고 도전적으로 몰입하여 성과를 창출했거나 목표를 달성한 경험을...,▶ 학생 목소리에 집중한 정책으로 참여율 70% 달성 ◀\n\n학부 시절 저희 학과...,1000,NaN,10월 28일


In [12]:
test[test['answer'].str.contains('아무리')]

,com,quest,answer,length,result,date
43,이베스트투자증권,"자신의 성격상 장단점은 무엇이며, 장점을 극대화하고 단점을 보완하기 위하여 어떠한 ...",▶넘치는 열정을 조절하다◀\n\n맡은 일이 아무리 힘들고 버거울지라도 끝까지 해내는...,500,x,10월 18일
70,게임빌컴투스플랫폼,주어진 일이나 과제 수행 시 새로운 것을 접목하거나 남다른 아이디어를 통해 문제를 ...,▶ 학생 목소리에 집중한 정책으로 참여율 70% 달성 ◀\n\n학부 시절 저희 학과...,0,NaN,10월 25일
87,SSG.COM,학업 외 가장 열정적이고 도전적으로 몰입하여 성과를 창출했거나 목표를 달성한 경험을...,▶ 학생 목소리에 집중한 정책으로 참여율 70% 달성 ◀\n\n학부 시절 저희 학과...,1000,NaN,10월 28일


In [30]:
a=test['answer'][test['answer'].str.contains('감성사전')]

list(a)

['▶1등으로 수료한 빅데이터 분석가 과정◀\n\n빅데이터 플랫폼을 활용한 수집, 저장, 처리, 분석까지의 단계와 머신러닝 기반 활용 및 시각화까지 공부하는 ‘PBL 머신러닝 기반 빅데이터 분석가 과정’을 1등으로 수료하였습니다. 교육과정 부반장을 맡아 매일 아침 데이터 분석 관련 뉴스를 정리해서 공유하고, 밤늦게까지 남아서 배운 내용을 스터디 멤버들과 서로 가르쳐주는 방식으로 부족한 부분을 채워가며 과정에 누구보다 열심히 임했습니다.\n\n▶ 감성사전을 통한 감성분류 정확도 94%달성◀\n\nㅇ에서 뉴스 빅데이터 분석을 통한 주가변동성 예측 프로젝트를 했던 경험이 있습니다. 신규투자자들의 투자의사결정을 지원해주는 뉴스데이터 분석 모델과 주가변동성 예측 모델을 만드는 것이 목표였습니다. 먼저, 주가 분석과 관련한 선행 연구 사례들 33개의 논문을 확인하였습니다. 그중에서도 참고할만한 논문 7개를 추려 기존 문제점들을 개선해나가는 식으로 방향을 잡았습니다. \n\n팀장을 맡아 Agile방식으로 프로젝트를 기획 및 총괄했으며, 중점적으로 제가 맡은 업무는 뉴스 데이터 분석으로 개선점을 크게 3가지로 잡았습니다. \n첫째, 데이터 양의 증가\n논문들의 한계의 공통점에는 활용한 데이터의 길이와 크기에 제한을 받았다는 것이었습니다. 따라서, 저희는 네이버 경제뉴스에 있는 모든 언론사의 기사 데이터를 수집 하여 총 2010년 1월부터 2021년 6월까지 약 11년치의 충분한 데이터를 확보하였습니다.\n둘째, 언어적 특성 고려\n저희는 BERT모델에 한국어를 학습시킨 KoBERT를 기반으로 감성분류 모델을 만들었습니다. BERT는 문장을 앞뒤로 해석하여 문맥을 이해할 수 있으므로 상황에 따라 달리 해석되는 다의어도 파악할 수 있는 특징이 있습니다. 이를 활용하여 문맥적 의미변이 및 동음이의어 구분의 문제를 해결하였습니다.\n셋째, 전문화된 감성사전\n기존 감성사전은 영화의 댓글이나 트위터 글에서 단순한 감정을 파악하는 것이었다면 저희가 만든 감성사전은 주식 도메인에 특화된

In [19]:
X=np.array(train.iloc[:,2:]).reshape(3125, 600, -1)

In [21]:
test_x=np.array(test.iloc[:,2:]).reshape(782, 600, -1)
test_x.shape

y = train_labels['label'].values
y = tf.keras.utils.to_categorical(train_labels['label']) 
y.shape

(3125, 61)

In [48]:
submission.to_csv('sub_chuchu.csv',index=False)